In [1]:
# Imports

import os
import re
from argparse import ArgumentParser

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib

from sklearn import preprocessing
from sklearn.model_selection import train_test_split

# PyTorch Lightning
import pytorch_lightning as pl
import seaborn as sns

# PyTorch
import torch
from torch import Tensor, nn, optim
import torch.nn.functional as F
import torch.utils.data as data

import torchmetrics.functional as metrics

from tqdm.notebook import tqdm
from IPython.display import set_matplotlib_formats
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.tuner.tuning import Tuner
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.loggers import WandbLogger, NeptuneLogger
from pytorch_tabular import TabularModel
from pytorch_tabular.models import CategoryEmbeddingModelConfig
from pytorch_tabular.config import DataConfig, OptimizerConfig, TrainerConfig, ExperimentConfig


import wandb

DEVICE = torch.device("cpu")

# Import GPU-related things
if torch.cuda.is_available():
    # import cupy as np
    # import cudf as pd

    # Ensure that all operations are deterministic on GPU (if used) for reproducibility
    torch.backends.cudnn.determinstic = True
    torch.backends.cudnn.benchmark = False

    DEVICE = torch.device("cuda:0")
# else:

# Plotting
plt.set_cmap("cividis")
#%matplotlib inline
set_matplotlib_formats("svg", "pdf")  # For export
matplotlib.rcParams["lines.linewidth"] = 2.0
sns.reset_orig()

# Path to the folder where the datasets are/should be downloaded (e.g. CIFAR10)
DATASET_PATH = os.environ.get("PATH_DATASETS", "data/")
# Path to the folder where the pretrained models are saved
CHECKPOINT_PATH = os.environ.get("PATH_CHECKPOINT", "saved_models/")

# Setting the seed
pl.seed_everything(42)

print('CUDA:', torch.cuda.is_available())
print("Device:", DEVICE)

/tmp/ipykernel_22734/30687496.py:53: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
  set_matplotlib_formats("svg", "pdf")  # For export
Global seed set to 42


CUDA: True
Device: cuda:0


<Figure size 640x480 with 0 Axes>

In [2]:
def in_ipython():
    try:
        return __IPYTHON__
    except NameError:
        return False

In [3]:
df = pd.read_csv('./data.csv')

In [4]:
X = df.drop(columns=['type', 'name'])
y = df['type']

X_encoder = preprocessing.LabelBinarizer()
y_encoder = preprocessing.LabelBinarizer()

name_trans = X_encoder.fit_transform(df['name'].to_numpy().reshape(-1, 1))
X = pd.concat([X, pd.DataFrame(name_trans)], axis=1)
y = y_encoder.fit_transform(y.to_numpy().reshape(-1, 1))

In [5]:
# Split into train+val and test
X_trainval, X_test, y_trainval, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Split train into train-val
X_train, X_val, y_train, y_val = train_test_split(X_trainval, y_trainval, test_size=0.1, random_state=0)

In [6]:
scaler = preprocessing.Normalizer()
X_train = scaler.transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

X_train, y_train = np.array(X_train), np.array(y_train)
X_val, y_val = np.array(X_val), np.array(y_val)
X_test, y_test = np.array(X_test), np.array(y_test)

/home/npiel2s/miniconda3/envs/mm/lib/python3.9/site-packages/sklearn/utils/validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/home/npiel2s/miniconda3/envs/mm/lib/python3.9/site-packages/sklearn/utils/validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/home/npiel2s/miniconda3/envs/mm/lib/python3.9/site-packages/sklearn/utils/validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


In [7]:
class CustomDataset(data.Dataset):
    def __init__(self, X, y):
        super().__init__()
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [8]:
def get_subfeatures(num_features, divider):
    return int(np.floor(num_features / divider))

In [9]:
class FFNetwork(pl.LightningModule):
    def __init__(self, num_features, num_classes, dropout: float = 0.1):
        super().__init__()

        self.num_features = num_features
        self.num_classes = num_classes

        self.save_hyperparameters()

        self.sequential = nn.Sequential(
            nn.Linear(num_features, num_features),
            nn.Linear(num_features, num_features),
            nn.Linear(num_features, num_features),
            nn.Linear(num_features, num_features),
            nn.Dropout(p=dropout),
            nn.ReLU(),
            nn.Linear(num_features, num_features),
            nn.Linear(num_features, num_features),
            nn.Linear(num_features, num_features),
            nn.Linear(num_features, num_features),
            nn.Dropout(p=dropout),
            nn.ReLU(),
            nn.Linear(num_features, num_features),
            nn.Linear(num_features, num_features),
            nn.Linear(num_features, num_features),
            nn.Linear(num_features, num_features),
            nn.Dropout(p=dropout),
            nn.ReLU(),
            nn.Linear(num_features, num_features),
            nn.Linear(num_features, num_features),
            nn.Linear(num_features, num_features),
            nn.Linear(num_features, num_features),
            nn.Dropout(p=dropout),
            nn.ReLU(),
            nn.Linear(num_features, num_features),
            nn.Linear(num_features, num_features),
            nn.Linear(num_features, num_features),
            nn.Linear(num_features, num_features),
            nn.Dropout(p=dropout),
            nn.ReLU(),
            nn.Linear(num_features, num_features),
            nn.Linear(num_features, num_features),
            nn.Linear(num_features, num_features),
            nn.Linear(num_features, num_features),
            nn.ReLU(),
            nn.Linear(num_features, num_classes),
            # nn.Softmax(dim=1)
        )

    def training_step(self, batch, batch_idx):
        loss = self._calculate_loss(batch, mode="train")
        return loss

    def validation_step(self, batch, batch_idx):
        _ = self._calculate_loss(batch, mode="val")

    def test_step(self, batch, batch_idx):
        _ = self._calculate_loss(batch, mode="test")

    def forward(self, X):
        return self.sequential(X)

    def _calculate_loss(self, batch, mode="train"):
        X, y = batch

        preds = self.forward(X)
        loss = F.cross_entropy(preds, y)

        # Logging to WANDB
        self.log(f"{mode}_loss", loss)
        self.log(f'{mode}_acc', metrics.accuracy(preds, y.long(), average='macro', num_classes=self.num_classes), prog_bar=True)
        self.log(f'{mode}_f1', metrics.f1_score(preds, y.long(), average='macro', num_classes=self.num_classes), prog_bar=True)
        self.log(f'{mode}_prc', metrics.precision(preds, y.long(), average='macro', num_classes=self.num_classes), prog_bar=False)
        self.log(f'{mode}_rcl', metrics.recall(preds, y.long(), average='macro', num_classes=self.num_classes), prog_bar=False)
        return loss

    def configure_optimizers(self):
        return optim.Adam(
            self.parameters(),
            lr=0.001,
            weight_decay=0.03
        )

    def train_dataloader(self):
        return data.DataLoader(
            CustomDataset(torch.from_numpy(X_train).float(), torch.from_numpy(y_train).float()),
            batch_size=64,
            shuffle=True,
            num_workers=8
        )

    def val_dataloader(self):
        return data.DataLoader(
            CustomDataset(torch.from_numpy(X_val).float(), torch.from_numpy(y_val).float()),
            batch_size=1,
            num_workers=8
        )

    def test_dataloader(self):
        return data.DataLoader(
            CustomDataset(torch.from_numpy(X_test).float(), torch.from_numpy(y_test).float()),
            batch_size=1,
            num_workers=8
        )

In [10]:
# train.py
def main(hparams):
    # wandb.finish()
    # wandb_logger = WandbLogger(project="bachelor")

    neptune_logger = NeptuneLogger(
        project="caigh/bachelor",
        api_key="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiI4NjQyYTAwMy1jMmJkLTRjMzctYWQ0Zi03Y2FmYjQ1YmQ2Y2MifQ==",
        log_model_checkpoints=True,
    )

    neptune_logger.finalize('success')

    print('Loading data..')
    print(f'X shape: {X.shape[1]}')
    print(f'y shape: {y.shape[1]}')

    model = FFNetwork(
        num_features=X.shape[1],
        num_classes=y.shape[1],
        dropout=0.5
    )

    # train the model
    trainer = pl.Trainer(
        accelerator='gpu',
        strategy='dp',
        precision=16,
        max_epochs=10,
        min_epochs=1,
        # overfit_batches=1,
        logger=neptune_logger,
        callbacks=[
            EarlyStopping(monitor="train_loss", mode="min"),
            ModelCheckpoint(dirpath='./checkpoints', filename='{epoch}-{val_loss:.2f}-{val_f1:.2f}', monitor='val_f1', save_last=True),
        ],
     )

    trainer.fit(model=model)

In [11]:
if __name__ == "__main__":
    if not in_ipython():
        root_dir = os.path.dirname(os.path.realpath(__file__))
        parser = ArgumentParser(add_help=False)
        hyperparams = parser.parse_args()

        # TRAIN
        main(hyperparams)
    else:
        main(None)

ModuleNotFoundError: You want to use the `Neptune` logger which is not installed yet, install it with `pip install neptune-client`.